In [ ]:
using Pkg
Pkg.instantiate()

In [ ]:
using Revise, CairoMakie
using ColonyImages
using DataFrames, CSV, FileIO, Dates, Serialization
import ColorSchemes as co 

In [ ]:
para = parameters(
        im_size = [1600,1600],
        time_points = [0:2.0:48...],
        colony_nr = 32);

simulations = ["Random", "Finger_weak", "Finger_strong"]
plot_sim = false

false

In [ ]:
fig2 =heatmap(img_int_vec_1[1][:,:,5], colormap = (:reds,0.5))
heatmap!(fig2.axis,conv(img_int_vec_1[1][:,:,5],laplac_kernel) .>0.1, colormap = (:blues, 0.5))
display(fig2)

LoadError: UndefVarError: `laplac_kernel` not defined

### Create empty test data for different methods below

In [97]:
test_array = [1 2 ; 3 1]
index = findall(test_array .>2)

1-element Vector{CartesianIndex{2}}:
 CartesianIndex(2, 1)

In [110]:
normalize(index[1].I.-[1,1])

2-element Vector{Float64}:
 1.0
 0.0

In [ ]:
findall(dir)

In [89]:
number_vecs = 20
dir= generate_dir_vec(20,0.3)

if 1 ∉ (dot.(dir,[[1,0] for i in length(dir)]) .>0.99)
    println(1)
end

In [2]:
img_int_vec = Vector{BitArray{3}}(undef, 0)
for i in 1:para.colony_nr
    hans =zeros(Bool, im_size...,stacks);
    hans = BitArray(hans)
    push!(img_int_vec, hans)
end
build_artifical_colony!(Center,(@view img_int_vec[1][:,:,1]),radius_circle,Points)
pixel_to_add = round(Int,sum(img_int_vec[1][:,:,1])).*(col_size_add_diff)

LoadError: UndefVarError: `para` not defined

### used different methods to calculate border area: 
first one is to use inbuild diff/gradient function of julia 
image needs to be cast on float and a lot of allocation have to be made 
-> not optimal, and also not entirely sure if it works in all "edge" :) cases


In [31]:
@benchmark begin 
    di2 = diff(Float64.(img_int_vec_1[1][:,:,1]), dims =2);
    di2 = hcat(di2, zeros(1600))
    di1 = diff(Float64.(img_int_vec_1[1][:,:,1]), dims =1);
    di1 = vcat(di1, zeros(1600)')
end


BenchmarkTools.Trial: 154 samples with 1 evaluation.
 Range (min … max):  27.589 ms … 43.595 ms  ┊ GC (min … max):  0.00% … 36.19%
 Time  (median):     34.649 ms              ┊ GC (median):    19.52%
 Time  (mean ± σ):   32.520 ms ±  3.407 ms  ┊ GC (mean ± σ):  14.17% ±  9.55%

  ▃                                                    ▁█▂     
  █▆▄▆▄▃▂▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▄▄▅███▆▅▄ ▂
  27.6 ms         Histogram: frequency by time        35.4 ms <

 Memory estimate: 117.80 MiB, allocs estimate: 30.

In [32]:
di2 = diff(Float64.(img_int_vec_1[1][:,:,1]), dims =2);
di2 = hcat(di2, zeros(1600))
di1 = diff(Float64.(img_int_vec_1[1][:,:,1]), dims =1);
di1 = vcat(di1, zeros(1600)')
display(heatmap(di2.+di1))

GLMakie.Screen(...)

second way is to use the package ImageEdgeDetection and its inbuild function imgradient. As runtime is similar to first approach, it appears it does similar things under the hood. 

In [39]:
using ImageEdgeDetection, MosaicViews, ImageFiltering, ImageCore

orientation_convention₁ = OrientationConvention(in_radians = false, compass_direction = 'S')

@benchmark begin 
    g1, g2 = imgradients(img_int_vec[1][:,:,1], KernelFactors.sobel)
    angles1 = detect_gradient_orientation(g1, g2, orientation_convention₁) / 360
    findall(angles1 .!=0)
end



BenchmarkTools.Trial: 66 samples with 1 evaluation.
 Range (min … max):  67.796 ms … 82.287 ms  ┊ GC (min … max):  0.00% … 15.81%
 Time  (median):     77.245 ms              ┊ GC (median):    10.64%
 Time  (mean ± σ):   76.566 ms ±  3.594 ms  ┊ GC (mean ± σ):  10.20% ±  4.32%

                                    ▂    █                     
  ▅▁▆▃▃▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▁▁▁▁██████▆█▅▅▃▅▁▃▁▁▁▁▃█▁▁▁▁▆ ▁
  67.8 ms         Histogram: frequency by time          82 ms <

 Memory estimate: 157.06 MiB, allocs estimate: 469.

Third way, is to use the distance and feature transform function of the package ImageMorphology which makes an inverse [Distance Transformation](https://en.wikipedia.org/wiki/Distance_transform) and to isolate the points with value 1 that are 1 gridpoint away from the colony. This approach is also the fastest by a factor of 2, but still requires a lot of allocations. Not sure if it can be optimised a lot. Current workaround is to only use it, after every x-added gridpoints. 

In [40]:
using ImageMorphology

@benchmark findall((distance_transform(feature_transform(img_int_vec[1][:,:,1])).== 1))

BenchmarkTools.Trial: 115 samples with 1 evaluation.
 Range (min … max):  40.709 ms … 49.351 ms  ┊ GC (min … max): 0.00% … 14.01%
 Time  (median):     41.550 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   43.679 ms ±  3.370 ms  ┊ GC (mean ± σ):  5.15% ±  6.65%

   ▂▇█▅▅ ▁                                            ▂        
  ▅█████▆█▄▄▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▇▇█▄▆▇▃▆▄ ▃
  40.7 ms         Histogram: frequency by time        49.2 ms <

 Memory estimate: 59.22 MiB, allocs estimate: 19.

In [41]:
fig1 = heatmap(distance_transform(feature_transform(img_int_vec[1][:,:,1])).== 1, colormap = (:blues, 0.8))
heatmap!(fig1.axis, img_int_vec[1][:,:,1], colormap = (:reds, 0.5))
display(fig1)

GLMakie.Screen(...)

Fourth way and probably best way, is to use convolution. The image is convoluted with a laplacian [kernel](https://sbme-tutorials.github.io/2018/cv/notes/4_week4.html) which only assumes values above zero on edges that are not occupied. The initial convultion operation using FFTW is quite slow, but as convolution is additative, changes in the image do not require to convoluted the image again, only the convolution of the added point with the kernel has to be added to the conv image. As a a single added point is the identity element of convolution, it gets even simpler and only the kernel has to be added at the positon of the added point to the convoluted image.  

In [46]:
laplac_kernel = [0 1 0; 1 -4 1; 0 1 0]

3×3 Matrix{Int64}:
 0   1  0
 1  -4  1
 0   1  0

In [48]:
@benchmark begin 
    test = conv(img_int_vec_1[1][:,:,1],laplac_kernel)
    test[800:802,800:802] += laplac_kernel
    findall(test.>0.1)
end


BenchmarkTools.Trial: 11 samples with 1 evaluation.
 Range (min … max):  473.998 ms … 497.046 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     477.292 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   478.636 ms ±   6.676 ms  ┊ GC (mean ± σ):  0.38% ± 0.66%

  █       ▃                                                      
  █▇▁▁▇▁▁▁█▁▁▇▁▇▁▁▁▁▁▁▁▁▁▇▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▇ ▁
  474 ms           Histogram: frequency by time          497 ms <

 Memory estimate: 59.34 MiB, allocs estimate: 33.

In [55]:
test = conv(img_int_vec_1[1][:,:,1],laplac_kernel)
display(heatmap(test .>0.1))

GLMakie.Screen(...)

All the research above condensed into to seperate functions, 'expand_colony_radom!' simulating a random growth using the distance transformation of the package ImageMorpohlogy which is quite linear in scaling (depending on the reshuffle counter). 'expand_colony_radom_cov!' simulated the same random growth using convolution and a laplace kernel, its runtime is very nonlinear as the inital convolution always takes around 400 ms but everything that follows takes much less time, therefore for many added pixels and a low reshuffle counter the `expand_colony_radom_cov!` function is much faster. 

In [54]:
@benchmark expand_colony_radom_cov!((@view img_int_vec_1[3][:,:,1]),pixel_to_add[1])

BenchmarkTools.Trial: 11 samples with 1 evaluation.
 Range (min … max):  483.926 ms … 514.292 ms  ┊ GC (min … max): 0.00% … 2.39%
 Time  (median):     489.690 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   492.199 ms ±   8.984 ms  ┊ GC (mean ± σ):  0.64% ± 0.90%

  ███ █  █   █   █    █   █       █                           █  
  ███▁█▁▁█▁▁▁█▁▁▁█▁▁▁▁█▁▁▁█▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  484 ms           Histogram: frequency by time          514 ms <

 Memory estimate: 64.43 MiB, allocs estimate: 8064.

In [50]:
@benchmark expand_colony_radom!((@view img_int_vec_1[4][:,:,1]),pixel_to_add[1])

BenchmarkTools.Trial: 4 samples with 1 evaluation.
 Range (min … max):  1.227 s …   1.403 s  ┊ GC (min … max): 4.50% … 4.32%
 Time  (median):     1.303 s              ┊ GC (median):    4.69%
 Time  (mean ± σ):   1.309 s ± 73.942 ms  ┊ GC (mean ± σ):  4.59% ± 0.24%

  █                 █           █                         █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  1.23 s         Histogram: frequency by time         1.4 s <

 Memory estimate: 1.38 GiB, allocs estimate: 363.

In [52]:
typeof(img_int_vec_1[3][:,:,1])

BitMatrix (alias for BitArray{2})

In [56]:
point = [740,745] .-Center

2-element Vector{Int64}:
 -60
 -55

In [64]:
dir = [-1,0]

2-element Vector{Int64}:
 -1
  0

In [63]:
dot(dir,point)

115

In [66]:
center = size(img_int_vec_1[3][:,:,1]).÷2

(800, 800)